<h1>2b. Machine Learning using tf.estimator </h1>

In this notebook, we will create a machine learning model using tf.estimator and evaluate its performance.  The dataset is rather small (7700 samples), so we can do it all in-memory.  We will also simply pass the raw data in as-is. 

In [1]:
import datalab.bigquery as bq
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

print(tf.__version__)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.8.0


Read data created in the previous chapter.

In [2]:
# In CSV, label is the first column, after the features, followed by the key
CSV_COLUMNS = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
FEATURES = CSV_COLUMNS[1:len(CSV_COLUMNS) - 1]
LABEL = CSV_COLUMNS[0]

df_train = pd.read_csv('./taxi-train.csv', header = None, names = CSV_COLUMNS)
df_valid = pd.read_csv('./taxi-valid.csv', header = None, names = CSV_COLUMNS)
df_test = pd.read_csv('./taxi-test.csv', header = None, names = CSV_COLUMNS)

<h2> Train and eval input functions to read from Pandas Dataframe </h2>

In [3]:
def make_train_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000
  )

In [4]:
def make_eval_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

Our input function for predictions is the same except we don't provide a label

In [5]:
def make_prediction_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = None,
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

### Create feature columns for estimator

In [6]:
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in FEATURES]
  return input_columns

<h3> Linear Regression with tf.Estimator framework </h3>

In [7]:
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model = tf.estimator.LinearRegressor(
      feature_columns = make_feature_cols(), model_dir = OUTDIR)

model.train(input_fn = make_train_input_fn(df_train, num_epochs = 10))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': 'worker', '_num_worker_replicas': 1, '_global_id_in_cluster': 0, '_keep_checkpoint_max': 5, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd8a20963c8>, '_is_chief': True, '_evaluation_master': '', '_task_id': 0, '_session_config': None, '_train_distribute': None, '_tf_random_seed': None, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_num_ps_replicas': 0, '_model_dir': 'taxi_trained', '_master': ''}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:step = 1, loss = 277

Evaluate on the validation data (we should defer using the test data to after we have selected a final model).

In [8]:
def print_rmse(model, df):
  metrics = model.evaluate(input_fn = make_eval_input_fn(df))
  print('RMSE on dataset = {}'.format(np.sqrt(metrics['average_loss'])))
print_rmse(model, df_valid)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-30-07:43:45
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-30-07:43:46
INFO:tensorflow:Saving dict for global step 608: average_loss = 115.46156, global_step = 608, loss = 13731.679
RMSE on dataset = 10.745304107666016


This is nowhere near our benchmark (RMSE of $6 or so on this data), but it serves to demonstrate what TensorFlow code looks like.  Let's use this model for prediction.

In [9]:
predictions = model.predict(input_fn = make_prediction_input_fn(df_test))
for items in predictions:
  print(items)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([9.056156], dtype=float32)}
{'predictions': array([9.054032], dtype=float32)}
{'predictions': array([9.054747], dtype=float32)}
{'predictions': array([9.052715], dtype=float32)}
{'predictions': array([9.055937], dtype=float32)}
{'predictions': array([9.055756], dtype=float32)}
{'predictions': array([9.054572], dtype=float32)}
{'predictions': array([9.054598], dtype=float32)}
{'predictions': array([9.056062], dtype=float32)}
{'predictions': array([9.054287], dtype=float32)}
{'predictions': array([9.056158], dtype=float32)}
{'predictions': array([9.056295], dtype=float32)}
{'predictions': array([9.050695], dtype=float32)}
{'predictions': array([9.053962], dtype=float32)}
{'predictions': array([9.102193], dty

{'predictions': array([9.0534115], dtype=float32)}
{'predictions': array([9.053725], dtype=float32)}
{'predictions': array([9.055162], dtype=float32)}
{'predictions': array([9.055378], dtype=float32)}
{'predictions': array([9.055987], dtype=float32)}
{'predictions': array([9.054003], dtype=float32)}
{'predictions': array([9.054873], dtype=float32)}
{'predictions': array([9.057184], dtype=float32)}
{'predictions': array([9.065873], dtype=float32)}
{'predictions': array([9.101553], dtype=float32)}
{'predictions': array([9.24311], dtype=float32)}
{'predictions': array([9.053308], dtype=float32)}
{'predictions': array([9.055083], dtype=float32)}
{'predictions': array([9.245744], dtype=float32)}
{'predictions': array([9.053659], dtype=float32)}
{'predictions': array([9.055489], dtype=float32)}
{'predictions': array([9.102037], dtype=float32)}
{'predictions': array([9.102657], dtype=float32)}
{'predictions': array([9.148609], dtype=float32)}
{'predictions': array([9.054611], dtype=float32)}


{'predictions': array([9.052523], dtype=float32)}
{'predictions': array([9.29143], dtype=float32)}
{'predictions': array([9.056605], dtype=float32)}
{'predictions': array([9.054148], dtype=float32)}
{'predictions': array([9.052364], dtype=float32)}
{'predictions': array([9.052282], dtype=float32)}
{'predictions': array([9.103003], dtype=float32)}
{'predictions': array([9.054458], dtype=float32)}
{'predictions': array([9.054958], dtype=float32)}
{'predictions': array([9.05376], dtype=float32)}
{'predictions': array([9.150516], dtype=float32)}
{'predictions': array([9.053902], dtype=float32)}
{'predictions': array([9.057131], dtype=float32)}
{'predictions': array([9.054162], dtype=float32)}
{'predictions': array([9.056339], dtype=float32)}
{'predictions': array([9.055281], dtype=float32)}
{'predictions': array([9.055566], dtype=float32)}
{'predictions': array([9.054192], dtype=float32)}
{'predictions': array([9.244525], dtype=float32)}
{'predictions': array([9.054742], dtype=float32)}
{'

{'predictions': array([9.054346], dtype=float32)}
{'predictions': array([9.05343], dtype=float32)}
{'predictions': array([9.055222], dtype=float32)}
{'predictions': array([9.054781], dtype=float32)}
{'predictions': array([9.054306], dtype=float32)}
{'predictions': array([9.053858], dtype=float32)}
{'predictions': array([9.054958], dtype=float32)}
{'predictions': array([9.05461], dtype=float32)}
{'predictions': array([9.0551405], dtype=float32)}
{'predictions': array([9.054724], dtype=float32)}
{'predictions': array([9.054542], dtype=float32)}
{'predictions': array([9.054525], dtype=float32)}
{'predictions': array([9.148577], dtype=float32)}
{'predictions': array([9.056782], dtype=float32)}
{'predictions': array([9.0539875], dtype=float32)}
{'predictions': array([9.102053], dtype=float32)}
{'predictions': array([9.195236], dtype=float32)}
{'predictions': array([9.049359], dtype=float32)}
{'predictions': array([9.05663], dtype=float32)}
{'predictions': array([9.056], dtype=float32)}
{'pr

{'predictions': array([9.053333], dtype=float32)}
{'predictions': array([9.054868], dtype=float32)}
{'predictions': array([9.055431], dtype=float32)}
{'predictions': array([9.089783], dtype=float32)}
{'predictions': array([9.103022], dtype=float32)}
{'predictions': array([9.10224], dtype=float32)}
{'predictions': array([9.102399], dtype=float32)}
{'predictions': array([9.055499], dtype=float32)}
{'predictions': array([9.05496], dtype=float32)}
{'predictions': array([9.150453], dtype=float32)}
{'predictions': array([9.10164], dtype=float32)}
{'predictions': array([9.233529], dtype=float32)}
{'predictions': array([9.102743], dtype=float32)}
{'predictions': array([9.052233], dtype=float32)}
{'predictions': array([9.243172], dtype=float32)}
{'predictions': array([9.055985], dtype=float32)}
{'predictions': array([9.102907], dtype=float32)}
{'predictions': array([9.054986], dtype=float32)}
{'predictions': array([9.055197], dtype=float32)}
{'predictions': array([9.051526], dtype=float32)}
{'p

{'predictions': array([9.056693], dtype=float32)}
{'predictions': array([9.054203], dtype=float32)}
{'predictions': array([9.05123], dtype=float32)}
{'predictions': array([9.10217], dtype=float32)}
{'predictions': array([9.054433], dtype=float32)}
{'predictions': array([9.102401], dtype=float32)}
{'predictions': array([9.097456], dtype=float32)}
{'predictions': array([9.195677], dtype=float32)}
{'predictions': array([9.046115], dtype=float32)}
{'predictions': array([9.09894], dtype=float32)}
{'predictions': array([9.193681], dtype=float32)}
{'predictions': array([9.054331], dtype=float32)}
{'predictions': array([9.150264], dtype=float32)}
{'predictions': array([9.054524], dtype=float32)}
{'predictions': array([9.102634], dtype=float32)}
{'predictions': array([9.053022], dtype=float32)}
{'predictions': array([9.054831], dtype=float32)}
{'predictions': array([9.101158], dtype=float32)}
{'predictions': array([9.055223], dtype=float32)}
{'predictions': array([9.093471], dtype=float32)}
{'p

This explains why the RMSE was so high -- the model essentially predicts the same amount for every trip.  Would a more complex model help? Let's try using a deep neural network.  The code to do this is quite straightforward as well.

<h3> Deep Neural Network regression </h3>

In [10]:
tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
model = tf.estimator.DNNRegressor(hidden_units = [32, 8, 2],
      feature_columns = make_feature_cols(), model_dir = OUTDIR)
model.train(input_fn = make_train_input_fn(df_train, num_epochs = 100));
print_rmse(model, df_valid)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': 'worker', '_num_worker_replicas': 1, '_global_id_in_cluster': 0, '_keep_checkpoint_max': 5, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd87cc9add8>, '_is_chief': True, '_evaluation_master': '', '_task_id': 0, '_session_config': None, '_train_distribute': None, '_tf_random_seed': None, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_num_ps_replicas': 0, '_model_dir': 'taxi_trained', '_master': ''}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:step = 1, loss = 472

We are not beating our benchmark with either model ... what's up?  Well, we may be using TensorFlow for Machine Learning, but we are not yet using it well.  That's what the rest of this course is about!

But, for the record, let's say we had to choose between the two models. We'd choose the one with the lower validation error. Finally, we'd measure the RMSE on the test data with this chosen model.

<h2> Benchmark dataset </h2>

Let's do this on the benchmark dataset.

In [11]:
import datalab.bigquery as bq
import numpy as np
import pandas as pd

def create_query(phase, EVERY_N):
  """
  phase: 1 = train 2 = valid
  """
  base_query = """
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  CONCAT(STRING(pickup_datetime), STRING(pickup_longitude), STRING(pickup_latitude), STRING(dropoff_latitude), STRING(dropoff_longitude)) AS key,
  DAYOFWEEK(pickup_datetime)*1.0 AS dayofweek,
  HOUR(pickup_datetime)*1.0 AS hourofday,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers,
FROM
  [nyc-tlc:yellow.trips]
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  """

  if EVERY_N == None:
    if phase < 2:
      # Training
      query = "{0} AND ABS(HASH(pickup_datetime)) % 4 < 2".format(base_query)
    else:
      # Validation
      query = "{0} AND ABS(HASH(pickup_datetime)) % 4 == {1}".format(base_query, phase)
  else:
    query = "{0} AND ABS(HASH(pickup_datetime)) % {1} == {2}".format(base_query, EVERY_N, phase)
    
  return query

query = create_query(2, 100000)
df = bq.Query(query).to_dataframe()

In [12]:
print_rmse(model, df)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-30-07:46:27
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-6071
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-30-07:46:27
INFO:tensorflow:Saving dict for global step 6071: average_loss = 88.98855, global_step = 6071, loss = 11310.672
RMSE on dataset = 9.433374404907227


RMSE on benchmark dataset is <b>9.41</b> (your results will vary because of random seeds).

This is not only way more than our original benchmark of 6.00, but it doesn't even beat our distance-based rule's RMSE of 8.02.

Fear not -- you have learned how to write a TensorFlow model, but not to do all the things that you will have to do to your ML model performant. We will do this in the next chapters. In this chapter though, we will get our TensorFlow model ready for these improvements.

In a software sense, the rest of the labs in this chapter will be about refactoring the code so that we can improve it.

## Challenge Exercise

Create a neural network that is capable of finding the volume of a cylinder given the radius of its base (r) and its height (h). Assume that the radius and height of the cylinder are both in the range 0.5 to 2.0. Simulate the necessary training dataset.
<p>
Hint (highlight to see):
<p style='color:white'>
The input features will be r and h and the label will be $\pi r^2 h$
Create random values for r and h and compute V.
Your dataset will consist of r, h and V.
Then, use a DNN regressor.
Make sure to generate enough data.
</p>

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License